In [11]:
R.<q> = LaurentPolynomialRing(ZZ)
W = CoxeterGroup('B4')
[s1,s2,s3,s4] = W.simple_reflections()



H = IwahoriHeckeAlgebra(W, q^(-1), -q)
T=H.T(); Cp=H.Cp(); C=H.C()
w0=W.long_element()

n = rank(W)
e = W(1)


def from_T_to_W(c):
    if c == 1: 
        return W(1)
    
    return W.from_reduced_word(  eval(str(c)[1:]) )
        
    


def working(x):
    result = {}
    
    for Delta in x.monomials():

        
        WDelta = from_T_to_W(Delta)
        
        pol = x.coefficient( WDelta  ) 
        
        pol_dict = pol.dict()
        for m in pol_dict:
            
            if m not in result.keys():
                result[m] = {}
            
            result[m][tuple(WDelta.reduced_word())] = pol_dict[m]
    
    result_ = GradedChar()
    result_.component = result
    return result_
        
    
class GradedChar:
    '''A class representing a graded character of module in the graded version of category O.
    self.graded is a dictionary where keys are indices of the graded components.
    Each graded component is a dictionary with keys being composition factors in that graded piece, and values are multiplicities.'''
    
    def __init__(self):
        self.component = {}
        self.name = ""
           
    def __str__(self):
        '''Each line becomes graded piece.'''
        
        if self == char_0():   # For zero character, print "0" instead of nothing.
            return "0"
        
        str_glob = ""
        if self.name != "":
            str_glob += self.name +":\n\n"
        
        for i in sorted(self.component.keys()):
            lis_i = []
            
            for w in (self.component[i]).keys():
                
                m = self.component[i][w]    # Multiplicity of w in i-th graded piece of self.
                m_string = ""
                if m != 1:
                    m_string = "%d*"%m      # Needed to print nicelly formated graded pieces.
                
                lis_i += ['%s'%m_string + str(w)]
            
            str_glob += "%d: "%i + ", ".join(lis_i) + "\n"
        
        return str_glob

    def __eq__(self, other): 
        if not isinstance(other, GradedChar):
            # Don't attempt to compare against unrelated types.
            return NotImplemented
        
        self.remove_zeros()      # Remove all the redundancies before comparing.
        other.remove_zeros()
        
        return self.component == other.component

    def __add__(self,char2):    
        char_sum = GradedChar()

        for k in (set(self.component.keys()) | set(char2.component.keys())):    # Union of keys without repeition.

            self_k = self.component.get(k,{})   # ".get" avoids KeyError
            char2_k = char2.component.get(k,{})

            char_sum.component[k] = {}

            for w in (set(self_k.keys()) | set(char2_k.keys())):

                char_sum.component[k][w] = self_k.get(w,0) + char2_k.get(w,0)

        return char_sum
    
    def __mul__(self,other):
        prod = GradedChar()

        for k in self.component:
            prod.component[k] = {}
        
            for w in self.component[k]:
                prod.component[k][w] = other * self.component[k][w]

        return prod
    
    def __rmul__(self,other):
        return self*other
    
    def __sub__(self,other):
        return self+((-1)*other)
    
    def __neg__(self):
        return (-1)*self
    
    
    def rename(self,new_name):
        '''Changes the name of self, and returns new_name.'''
        
        self.name = new_name
        return self.name
    
    
    def remove_zeros(self):
        '''Removes all composition factors with coefficients zero.
        These can occur only when subtracting, i.e. dealing with characters of virtual modules.'''
        
        for k in self.component:
            remove_keys_with_value(self.component[k],0)
        
        remove_keys_with_value(self.component,{})
        
        return self
        
    
    def only_positive(self):
        '''Removes all composition factors with non-positive coefficients.
        These can occur only when subtracting, i.e. dealing with characters of virtual modules.'''
        
        for k in self.component:
            remove_keys_with_value_smaller_than(self.component[k],0)
        
        remove_keys_with_value(self.component,{})
        
        return self    
    
    
    def add_factor(self,i,w):
        '''Adds a composition factor to self in i-th graded piece.'''
        
        if i not in self.component:
            self.component[i] = {}
        
        if w not in self.component[i]:
            self.component[i][w] = 0
        
        self.component[i][w] += 1
        
        self.name = ""    # The character "self" has changed, so the old name is not valid anymore.
        
        return self

    
    def is_simple(self):
        '''Checks whether self is a simple module.'''

        self.remove_zeros()

        if len(components(self)) != 1:
            return False

        only_component = self.component[ list(self.component.keys())[0] ]

        if len(only_component.keys()) != 1:
            return False

        only_factor = list(only_component.keys())[0]

        if only_component[ only_factor ]  != 1:
            return False

        return True
   
    
    def is_true_character(self):
        '''Returns True iff all the coefficients of self are non-negative.
        Assumes that all the coeffictients are integers.'''
        
        for i in self.component:
            for w in self.component[i]:
                if not ( self.component[i][w] >= 0 ):
                    return False
        return True
  

    def min_character(self,char2):
        '''Returns the character whose components have multiplicities equal to minimum
        of the multiplicities of the corresponding components in self and char2.'''
        
        char_min = GradedChar()
       
        for k in (set(self.component.keys()) | set(char2.component.keys())):    # Union of keys without repeition.

            self_k = self.component.get(k,{})   # ".get" avoids KeyError
            char2_k = char2.component.get(k,{})

            char_min.component[k] = {}

            for w in (set(self_k.keys()) | set(char2_k.keys())):

                char_min.component[k][w] = min(self_k.get(w,0),char2_k.get(w,0))

        return char_min    
    

    
######### Some operations on graded characters #########


def shift(char,i):
    '''Returns the shift by i of char.'''
    
    char_new = GradedChar()
    
    for key in char.component.keys():
        char_new.component[key-i] = char.component[key]
    
    return char_new


def dual(char):
    '''Returns the graded dual of char.
    Input: GradedChar.'''
    
    char_dual = GradedChar()
    
    for key in char.component.keys():
       char_dual.component[-key] = char.component[key]
    
    return char_dual


def components(char):
    '''Returns the list of indexes of non-zero graded pieces of char.'''
    
    char.remove_zeros()
    return sorted(char.component.keys())


def truncate(char,lis):
    '''Cuts off from char all graded pieces from lis.'''
    
    for k in lis:
        char.component.pop(k, None)
    
    return char
    
    
def mult(char,i,w):
    '''Returns the multiplicity of w in i-th graded piece of char.'''
    
    if i not in char.component:
        return 0
    
    return char.component[i].get(w,0)


def total_mult(char,w):
    '''Returns the total multiplicity of w in char.'''
    
    return sum(char.component[i].get(w,0) for i in char.component)


def dict_mult(char,w):
    '''Returns dictionary with items (k:m), where m is the multiplicity of w in k-th graded piece of char.'''
    
    dic = {}
    for k in char.component:
        if char.component[k].get(w,0) != 0:
            dic[k] = char.component[k][w]
            
    return dic


def clean(char):
    '''Returns cleaner output: simple reflections are denoted by 1,2,3, ...'''
    
    char_dummy = GradedChar()
    
    char_dummy.name = (char.name).replace("(1)","(e)").replace("s","").replace("*","")
    
    for k in char.component:
        char_dummy.component[k] = {}
        
        for w in char.component[k]:
            char_dummy.component[k][convert_to_123(w)] = char.component[k][w]
    
    return(str(char_dummy))


def print_clean(char):
    '''Prints cleaner output: simple reflections are denoted by 1,2,3, ...'''
    
    print(clean(char))
    

def combinatorial_cokernel(char1, char2):
    '''Returns the lower bound on the cokernel of a map from X to Y.'''
    
    virtual = char2-char1
    virtual.only_positive()
    
    return virtual


def combinatorial_kernel(char1, char2):
    '''Returns the lower bound on the kernel of a map from X to Y.'''
    
    virtual = char1-char2
    virtual.only_positive()
    
    return virtual



def ungrade(X):
    '''Returns the ungraded version of graded character X: all composition factors are moved to degree 0.'''
    
    unX = char_0()
    
    for i in components(X):
        unX += shift(X,i)
    
    truncate(unX,[i for i in components(unX) if i != 0])
    
    return unX


def remove_keys_with_value(dict,value):
    '''Removes all items from dict with given value. Changes the original dict, does not return anything.
    Auxiliary function used in GradedChar().remove_zeros.'''
    
    remove = []
    for x in dict:
        if dict[x] == value:
            remove.append(x)
            
    for x in remove:
        dict.pop(x)
        
        
def remove_keys_with_value_smaller_than(dict,bound):
    '''Removes all items from dict with value smaller than or equal to "bound".
    Changes the original dict, does not return anything.
    Auxiliary function used in GradedChar().only_positive.'''
    
    remove = []
    for x in dict:
        if dict[x] <= bound:
            remove.append(x)
            
    for x in remove:
        dict.pop(x)


def proj_resolution_param(X,t):
    '''Returns the parameters of the combinatorial projective resolution of a graded character X.
    Here t is a bound on the length of the resolution, to avoid to long computations.
    Output is a list, and the elements are
    dictionaries with keys parameters (w,shift) and values are multiplicities.
    Algorithm is given by Hankyung.
    TODO: Make t optional variable.'''
   
    resolution_param = []

    while X != GradedChar():

        P_param = {}   # to be the set of parameters in the projective resolution
        
        X1 = GradedChar()  # to be the next X
        C = copy(X)

        while C != GradedChar():
            
            top_P = GradedChar()
            top_deg = min(components(C))
            for w in C.component[top_deg]:
                
                top_P += C.component[top_deg][w] * shift(working(T(Cp[w])),-top_deg)   
                
                if (w,-top_deg) in P_param:
                    P_param[(w,-top_deg)] += C.component[top_deg][w]
                else:
                    P_param[(w,-top_deg)] = C.component[top_deg][w]
                

            K = combinatorial_kernel(top_P,C)    
            C = combinatorial_cokernel(top_P,C)

            X1 += K
        
        resolution_param.append(P_param)
        
        if len(resolution_param)>t+1: # Here exit if too big.
            return [False]*1000
        
        X = X1
    
    return resolution_param


def convert_to_123(w):
    '''Converts an element from W to the "123" string notation.
    Does not work with coefficients, as "convert_to_123_long".'''
    
    if w == W(1):
        return "e"
    
    return "".join([str(t) for t in list(w.reduced_word())])


def convert_to_123_v2(w):
    '''Converts an element from W to the "123" string notation.
    Does not work with coefficients, as "convert_to_123_long".'''
    
    if w == ():
        return "e"
    
    return "".join([str(t) for t in list(w)])


def res_cox_to_123(res):
    new_res = []
    
    for P in res:
        new_P = {}
        for f in P:
            new_P[( convert_to_123_v2(f[0]) , f[1] )] = P[f]
        new_res.append(new_P)
        
    return new_res


def proj_dim(X):

    return len(proj_resolution_param( working(   X  ) ,100))-1

In [12]:
typ = str(W.coxeter_type())[-6] + str(W.coxeter_type())[-2]

output = "# Type: %s\n"%typ
output += "# Formatting: y x D_comb_proj_res of theta_y Delta_x\n\n"

M=len(W)^2
i=0

for y in W:
    for x in W:
        P = proj_resolution_param( working(   T(T[x]*Cp[y])  ) ,100)
        output += "%s %s %s\n"%(convert_to_123(y), convert_to_123(x), res_cox_to_123(P) )
    
        if i%100 == 0:
            print("%d/%d"%(i,M))
        i += 1
    
#print(output)

text_file = open("Graded_characters_in_O_0_data/miscellaneous/%s_comb_proj_res.txt"%typ, "w")
n = text_file.write(output)
text_file.close()


0/147456
100/147456
200/147456
300/147456


KeyboardInterrupt: 

In [13]:
print(output)

# Type: B4
# Formatting: y x D_comb_proj_res of theta_y Delta_x

e e [{('e', 0): 1}]
e 1 [{('1', 0): 1}, {('e', -1): 1}]
e 2 [{('2', 0): 1}, {('e', -1): 1}]
e 3 [{('3', 0): 1}, {('e', -1): 1}]
e 4 [{('4', 0): 1}, {('e', -1): 1}]
e 12 [{('12', 0): 1}, {('2', -1): 1, ('1', -1): 1}, {('e', -2): 1}]
e 21 [{('21', 0): 1}, {('2', -1): 1, ('1', -1): 1}, {('e', -2): 1}]
e 23 [{('23', 0): 1}, {('2', -1): 1, ('3', -1): 1}, {('e', -2): 1}]
e 31 [{('31', 0): 1}, {('3', -1): 1, ('1', -1): 1}, {('e', -2): 1}]
e 32 [{('32', 0): 1}, {('2', -1): 1, ('3', -1): 1}, {('e', -2): 1}]
e 34 [{('34', 0): 1}, {('3', -1): 1, ('4', -1): 1}, {('e', -2): 1}]
e 41 [{('41', 0): 1}, {('1', -1): 1, ('4', -1): 1}, {('e', -2): 1}]
e 42 [{('42', 0): 1}, {('2', -1): 1, ('4', -1): 1}, {('e', -2): 1}]
e 43 [{('43', 0): 1}, {('3', -1): 1, ('4', -1): 1}, {('e', -2): 1}]
e 121 [{('121', 0): 1}, {('12', -1): 1, ('21', -1): 1}, {('2', -2): 1, ('1', -2): 1}, {('e', -3): 1}]
e 123 [{('123', 0): 1}, {('12', -1): 1, ('31', -1): 1, ('